In [2]:
# 📌 Project: Walkable Access to Bus Stops using Buffer Analysis

# --- Step 1: Import Required Libraries ---
import geopandas as gpd
import pandas as pd
import folium
from shapely.geometry import Point

# --- Step 2: Load Your GeoJSON Data ---
try:
    study_area = gpd.read_file("Study Area.geojson")
    road_links = gpd.read_file("Road.geojson")
    bus_stops = gpd.read_file("Bus Stop.geojson")
    land_use = gpd.read_file("Landuse.geojson")
    bus_route = gpd.read_file("Bus Route.geojson")  # Optional
    print("GeoJSON files loaded successfully.")
except Exception as e:
    print("Error loading files:", e)

# Optional: Check first few rows
print(study_area.head())
print(bus_stops.head())
print(land_use.head())

# --- Step 3: Coordinate Reference System ---
# Original CRS is likely EPSG:4326; reproject to metric for buffer
bus_stops_proj = bus_stops.to_crs(epsg=3395)

# Create 400m buffer
bus_buffer = bus_stops_proj.buffer(400)

# Convert buffer back to WGS84 for mapping
bus_buffer_wgs = gpd.GeoDataFrame(geometry=bus_buffer, crs="EPSG:3395").to_crs(epsg=4326)

# --- Step 4: Intersect Buffer with Study Area ---
study_area_proj = study_area.to_crs(epsg=4326)
intersected_area = gpd.overlay(study_area_proj, bus_buffer_wgs, how='intersection')

# --- Step 5: Estimate Population Covered ---
# Make sure study area has 'population' field
if 'population' in intersected_area.columns:
    pop_covered = intersected_area['population'].sum()
    print(f"Total population within 400m of a bus stop: {pop_covered}")
else:
    print("Column 'population' not found in study area.")

# --- Step 6: Intersect Buffer with Land Use ---
land_use_proj = land_use.to_crs(epsg=4326)
land_use_clipped = gpd.overlay(land_use_proj, bus_buffer_wgs, how='intersection')

# Optional: Analyze land use types
if 'land_use_type' in land_use_clipped.columns:
    land_use_summary = land_use_clipped['land_use_type'].value_counts()
    print("Land use within 400m buffer:")
    print(land_use_summary)
else:
    print("Column 'land_use_type' not found in land use data.")

# --- Step 7: Visualize in Folium ---
m = folium.Map(location=[bus_stops.geometry.y.mean(), bus_stops.geometry.x.mean()], zoom_start=13)

# Add Study Area
folium.GeoJson(study_area_proj, name="Study Area").add_to(m)

# Add Road Links
folium.GeoJson(road_links, name="Roads", style_function=lambda x: {'color': 'black', 'weight': 1}).add_to(m)

# Add Buffer Zones
folium.GeoJson(bus_buffer_wgs, name="400m Buffers", style_function=lambda x: {'color': 'blue', 'fillOpacity': 0.2}).add_to(m)

# Clean and Add Bus Stops (handle null geometries)
bus_stops = bus_stops[bus_stops.geometry.notnull()]
bus_stops = bus_stops[bus_stops.geometry.type == 'Point']

for idx, row in bus_stops.iterrows():
    folium.Marker([row.geometry.y, row.geometry.x], popup=f"Bus Stop {idx+1}").add_to(m)

# Add Land Use within Buffer
folium.GeoJson(land_use_clipped, name="Land Use in Buffer", style_function=lambda x: {'color': 'green', 'fillOpacity': 0.3}).add_to(m)

# Optionally add Bus Route
if bus_route is not None:
    folium.GeoJson(bus_route, name="Bus Route", style_function=lambda x: {'color': 'red', 'weight': 2}).add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

# Save and show output
m.save("buffer_analysis_map.html")
print("✅ Interactive map saved as 'buffer_analysis_map.html'")


C:\Users\ANY2024\AppData\Local\Programs\Python\Python313\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: Several features with id = 1 have been found. Altering it to be unique. This warning will not be emitted anymore for this layer
  return ogr_read(
C:\Users\ANY2024\AppData\Local\Programs\Python\Python313\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: Several features with id = 1 have been found. Altering it to be unique. This warning will not be emitted anymore for this layer
  return ogr_read(


GeoJSON files loaded successfully.
   new ID   poly_type     layer  area_acre  population  \
0     102  peripheral  External   71078.76     1457115   
1     103  peripheral  External   13451.77      275761   
2     104  peripheral  External  126188.07     2586855   
3       1        core  Internal    3992.92       81855   
4       2        core  Internal    1982.74       40646   

                                            geometry  
0  MULTIPOLYGON Z (((101.48137 3.06001 0, 101.480...  
1  MULTIPOLYGON Z (((101.6189 3.19509 0, 101.6188...  
2  MULTIPOLYGON Z (((101.49287 3.23053 0, 101.492...  
3  MULTIPOLYGON Z (((101.5986 3.14987 0, 101.5976...  
4  MULTIPOLYGON Z (((101.61712 3.21603 0, 101.616...  
   id                             Location layer  \
0   1           MRT Kwasa Sentral, Pintu A  T801   
1   2  Taman Industri Sungai Buloh (Timur)  T801   
2   3            Bistari De Kota (Selatan)  T801   
3   4              Bistari De Kota (Timur)  T801   
4   5                     